# Attributes of Winning Teams
## Exploring the Data of the EU Soccer League

The goal of this analysis will be to see if there are common attributes amoung the top performing leagues in the European Soccer League. Data is from a Kaggle post,[European Soccer Database](https://www.kaggle.com/hugomathien/soccer) by Hugomathien.

The process of discovery will be the following
1. Create SQL Queries to gather/organize original data
    * Which teams the players belong too (most recent year)
    * Join the player attributes to those teams (most recent performance scores)
    * Which teams placed top on the league (most recent year)
2. Read the SQL Queries into Pandas for further manipulation
3. Provide exploratory visualization showing the combined attributes of each top-performing team
4. Make a judgement if there are common attributes that matter most to success

## Setting Up Environment

First I want to make sure I have my enviroments set up. I have a work station with three monitors but I am not always there so I want to make sure I can do analysis on the fly on my Chromebook. To do this I have set up a cloud instance with [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). I started the project on my workstation and then createe my [GitHub Repo](https://github.com/GarrettMarkScott/European_Soccer_Exploring). In the process of doing this I quickly learned that the 300mb database (SQLite) was too large for a simple git push command. This forced me to create a .gitignore file that I placed the database in. I then was able to clone the git repo into my Jupyter Cloud Instance and simply upload the SQLite database into the same directory in my cloud instance. Now I can push/pull from both machines without large file transfers.

I used the below code to check to make sure my environment was working.

In [117]:
#These are the general items for simple loading

import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import pprint

conn = sqlite3.connect('soccer_data.sqlite')
tables = pd.read_sql('SELECT * FROM sqlite_master WHERE type="table";', conn)
countries = pd.read_sql_query('SELECT * FROM Country;', conn)
players = pd.read_sql_query('SELECT player_api_id, player_name FROM Player;', conn)
player_att = pd.read_sql_query('SELECT * FROM Player_Attributes;',conn)
player_att.columns.tolist()

['id',
 'player_fifa_api_id',
 'player_api_id',
 'date',
 'overall_rating',
 'potential',
 'preferred_foot',
 'attacking_work_rate',
 'defensive_work_rate',
 'crossing',
 'finishing',
 'heading_accuracy',
 'short_passing',
 'volleys',
 'dribbling',
 'curve',
 'free_kick_accuracy',
 'long_passing',
 'ball_control',
 'acceleration',
 'sprint_speed',
 'agility',
 'reactions',
 'balance',
 'shot_power',
 'jumping',
 'stamina',
 'strength',
 'long_shots',
 'aggression',
 'interceptions',
 'positioning',
 'vision',
 'penalties',
 'marking',
 'standing_tackle',
 'sliding_tackle',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_positioning',
 'gk_reflexes']

## Matching Players to Teams

Next was time to create the SQL queries to organize the raw data a little bit so it was easier to work with. I was able to borrow my girlfriends mac and load the SQLite datebase into [DB Browser for SQLite] for quick SQL Queries (since chromebook can't run programs).

I spent quite a bit of time trying to figure out how to merge the players to teams. As seen in the screenshot below there was not a common key between player and team. My assumption is that this is due to players switching various teams over their career.

![Table Screenshot](https://i.imgur.com/KCpqgS5.jpg)

I checked the *player_fifa_api_id* but that had 11,060 DISTINCT rows. I was able to create the query below that used the Match table data to see which players were associated with each match. Due to there being substitutions throughout the match, it is assumed that these 11 players are the starting roster for each team during the 2007-2008 season (May-August). 

This turned out to be a pretty big challege because I did not relize DB Browser for SQLite seemingly does not have a formal Date type but rather stores date types as a *Timestring* format. If anyone has any information regarding this and how to deal with it in the best/cleanest way I have made a [Stack Overflow Post](https://stackoverflow.com/questions/56202599/converting-timestrings-to-datetime-db-browser)

In [55]:
#Matches from most recent season and the starting players (in ID format)
matches = pd.read_sql_query('''SELECT t.team_long_name, m.home_player_1, m.home_player_2, m.home_player_3,m.home_player_4, m.home_player_5,m.home_player_6, m.home_player_7, m.home_player_8, m.home_player_9, m.home_player_10, m.home_player_11, m.date 
                                 FROM Team t 
                                 JOIN Match m ON t.team_api_id = m.home_team_api_id 
                                WHERE DATE(m.date) BETWEEN "2015-05-01" and "2016-05-25" 
                                ORDER BY t.team_long_name;''', conn)
matches.set_index('team_long_name', inplace=True)
matches

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,date
team_long_name,,,,,,,,,,,,
1. FC Köln,212815.0,36395.0,36934.0,264221.0,307210.0,36086.0,167589.0,127945.0,459493.0,166449.0,196366.0,2015-05-10 00:00:00
1. FC Köln,212815.0,36395.0,36934.0,264221.0,22824.0,450976.0,36086.0,127945.0,459493.0,196366.0,166449.0,2015-05-23 00:00:00
1. FC Köln,212815.0,127945.0,231753.0,303800.0,307210.0,166449.0,36086.0,167589.0,459493.0,177941.0,71605.0,2015-10-31 00:00:00
1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,166449.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-11-21 00:00:00
1. FC Köln,212815.0,127945.0,36934.0,303800.0,307210.0,238438.0,36086.0,167589.0,260470.0,177941.0,71605.0,2015-12-05 00:00:00
1. FC Köln,212815.0,212377.0,231753.0,36934.0,303800.0,307210.0,450976.0,36086.0,22824.0,127945.0,238438.0,2015-12-19 00:00:00
1. FC Köln,212815.0,212377.0,36934.0,303800.0,307210.0,167589.0,36086.0,127945.0,166449.0,260470.0,71605.0,2016-01-23 00:00:00
1. FC Köln,212815.0,212377.0,231753.0,303800.0,307210.0,127945.0,167589.0,36086.0,260470.0,238438.0,71605.0,2015-08-22 00:00:00
1. FC Köln,212815.0,36934.0,28435.0,303800.0,127945.0,450976.0,307210.0,259439.0,238438.0,260470.0,71605.0,2016-02-13 00:00:00


## Connecting Player ID to Player Name
To keep our code modular as possible the function below can be used to return a player's name based on ID input.

In [64]:
#Build a for loop that builds Teams Dictionary with team as the key and players as the values

def player_id_to_name(id): 
    return(players[players['player_api_id'] == id].iloc[0,1])

player_id_to_name(212815)

'Timo Horn'

## Creating Compound Dictionary of Teams, Players, & Attributes

Build a for loop that builds Teams Dictionary with team as the key and players as the values

This is our goal:
```
teams = {"team 1": {"Player 1": {"Dribbling": 4,
                                 "Attack": 5,
                                 "Speed": 9},
                    "Player 2": {"Dribbling": 6,
                                 "Attack": 10,
                                 "Speed": 3},
                    "Player 3": {"Dribbling": 2,
                                 "Attack": 5,
                                 "Speed": 8},
                    },
         "team 2": {"Player 1": {"Dribbling": 9,
                                 "Attack": 6,
                                 "Speed": 2},
                    "Player 2": {"Dribbling": 10,
                                 "Attack": 2,
                                 "Speed": 8},
                    "Player 3": {"Dribbling": 5,
                                 "Attack": 5,
                                 "Speed": 5},
                    },
```

In [105]:
teams = []
for team_name in matches.index:
    print(team_name)
#matches.index.tolist()

1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FC Köln
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
1. FSV Mainz 05
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
ADO Den Haag
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Monaco
AS Saint-Étienne
AS Saint-Étienne
AS Saint-

Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Pogoń Szczecin
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Polonia Bytom
Queens Park Rangers
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Celta de Vigo
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña
RC Deportivo de La Coruña

In [110]:


matches.loc['Zagłębie Lubin']
for i in matches:
     print(ivalue)



AttributeError: 'str' object has no attribute 'value'

In [103]:
#Chunk that gets names of home team players
for player in matches.loc[['Zagłębie Lubin'],[0]]:
    print(player_id_to_name(player))

KeyError: 'None of [[0]] are in the [columns]'